In [ ]:
from google.colab import drive
import os
drive.mount('/content/gdrive')

!ls /content/gdrive/MyDrive/aicup_2020
!unzip /content/gdrive/MyDrive/aicup_2020/Data_Description.zip
!unzip /content/gdrive/MyDrive/aicup_2020/trainset.zip
!unzip /content/gdrive/MyDrive/aicup_2020/testset.zip
!mkdir model
!ls

Mounted at /content/gdrive
Data_Description.zip  result_1245.csv  result_1258.csv	result_1271.csv
model		      result_1246.csv  result_1259.csv	result_1272.csv
result_1234.csv       result_1247.csv  result_1260.csv	result_1273.csv
result_1235.csv       result_1248.csv  result_1261.csv	result_1274.csv
result_1236.csv       result_1249.csv  result_1262.csv	result_1275.csv
result_1237.csv       result_1250.csv  result_1263.csv	result_1276.csv
result_1238.csv       result_1251.csv  result_1264.csv	result_1277.csv
result_1239.csv       result_1252.csv  result_1265.csv	result_1278.csv
result_1240.csv       result_1253.csv  result_1266.csv	result_1279.csv
result_1241.csv       result_1254.csv  result_1267.csv	result_1280.csv
result_1242.csv       result_1255.csv  result_1268.csv	result_1281.csv
result_1243.csv       result_1256.csv  result_1269.csv	testset.zip
result_1244.csv       result_1257.csv  result_1270.csv	trainset.zip
Archive:  /content/gdrive/MyDrive/aicup_2020/Data_Description.zip


In [1]:
!pip --freeze


Usage:   
  pip3 <command> [options]

no such option: --freeze


In [ ]:
!cat Description.md

## Data Description
* ### Id
    Serial number.
    流水編號，無特別意義。
* ### Title
    Title of the paper.
    論文文章標題。
* ### Abstract
    Abstract of the paper. Each sentence is be seperated by **$$$**.
    論文摘要，以 **$$$** 將句子隔開。
 
* ### Classifications
    Multilabel of the paper. Each label is be seperated by **space**.
    論文的分類，多個分類以 **空格** 做分割。


In [ ]:
!pip install transformers

     |████████████████████████████████| 1.5MB 11.0MB/s 
     |████████████████████████████████| 2.9MB 51.1MB/s 
     |████████████████████████████████| 890kB 50.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=05fcb995cc4b82a32022909592af8451640e4abad78f2c07533690d80fa22798
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from transformers import BertTokenizer, BertModel
from transformers import XLNetTokenizer, XLNetModel
from transformers import RobertaTokenizer, RobertaModel
from transformers import AdamW, get_linear_schedule_with_warmup

import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold
from tqdm.notebook import tqdm
import csv
import random
import matplotlib.pyplot as plt
import sys

use_cuda = torch.cuda.is_available()
if use_cuda:
    print("using cuda!")
device = torch.device("cuda" if use_cuda else "cpu")

using cuda!


In [ ]:
col_names = []
raw_data = []

with open("trainset.csv") as f:
    rows = csv.reader(f)
    for i, row in enumerate(rows):
        if i == 0:
            col_names = row
            continue
        raw_data.append(row)

print(raw_data[0])

['Distinctiveness, complexity, and repeatability of online signature templates', 'This paper proposes three measures to quantify the characteristics of online signature templates in terms of distinctiveness, complexity and repeatability.$$$A distinctiveness measure of a signature template is computed from a set of enrolled signature samples and a statistical assumption about random signatures.$$$Secondly, a complexity measure of the template is derived from a set of enrolled signature samples.$$$Finally, given a signature template, a measure to quantify the repeatability of the online signature is derived from a validation set of samples.$$$These three measures can then be used as an indicator for the performance of the system in rejecting random forgery samples and skilled forgery samples and the performance of users in providing accepted genuine samples, respectively.$$$The effectiveness of these three measures and their applications are demonstrated through experiments performed on 

In [ ]:
categories = {"THEORETICAL": 0, "ENGINEERING": 1, "EMPIRICAL": 2, "OTHERS": 3}

for row in raw_data:
    row[1] = row[1].replace("$$$", " ")
    row[2] = row[2].split()
    tmp = [0, 0, 0, 0]
    for cat in row[2]:
        tmp[categories[cat]] = 1
    row.remove(row[2])
    if tmp == [1, 0, 0, 0]:
        tmp = [0] + tmp
    elif tmp == [0, 1, 0, 0]:
        tmp = [1] + tmp
    elif tmp == [0, 0, 1, 0]:
        tmp = [2] + tmp
    elif tmp == [1, 1, 0, 0]:
        tmp = [3] + tmp
    elif tmp == [1, 0, 1, 0]:
        tmp = [4] + tmp
    elif tmp == [0, 1, 1, 0]:
        tmp = [5] + tmp
    elif tmp == [1, 1, 1, 0]:
        tmp = [6] + tmp
    else:
        tmp = [7] + tmp
    assert len(tmp) == 5
    row += tmp
print(raw_data[0])

['Distinctiveness, complexity, and repeatability of online signature templates', 'This paper proposes three measures to quantify the characteristics of online signature templates in terms of distinctiveness, complexity and repeatability. A distinctiveness measure of a signature template is computed from a set of enrolled signature samples and a statistical assumption about random signatures. Secondly, a complexity measure of the template is derived from a set of enrolled signature samples. Finally, given a signature template, a measure to quantify the repeatability of the online signature is derived from a validation set of samples. These three measures can then be used as an indicator for the performance of the system in rejecting random forgery samples and skilled forgery samples and the performance of users in providing accepted genuine samples, respectively. The effectiveness of these three measures and their applications are demonstrated through experiments performed on three onli

In [ ]:
ratios = [0.0, 0.0, 0.0, 0.0]
base = [0.0, 0.0, 0.0, 0.0]

for i in range(-4, 0):
    ratios[i] = sum([x[i] for x in raw_data]) / len(raw_data)
    base[i] = 2*ratios[i] / (1+ratios[i])
    
ratios, base

([0.4595714285714286, 0.4847142857142857, 0.30557142857142855, 0.037],
 [0.6297347557991583,
  0.652939478495141,
  0.46810373126162597,
  0.07135969141755062])

In [ ]:
class Data():
    
    def __init__(self, idxs, is_test):
        
        self.title = []
        self.sent = []
        self.label = []
        self.idxs = idxs
        self.is_test = is_test
    
    def readData(self, data_path):
        
        with open(data_path) as fd:
            rows = csv.reader(fd)
            for i, row in enumerate(rows, -1):
                if i == -1:
                    continue
                elif i in self.idxs:
                    self.title.append(row[1 if self.is_test else 0])
                    self.sent.append(row[2 if self.is_test else 1].replace("$$$", " "))
                    if not self.is_test:
                        cates = row[2].split()
                        tmp = [0, 0, 0, 0]
                        for cate in cates:
                            tmp[categories[cate]] = 1
                        self.label.append(tmp)
        
    def batchData(self, batch_size):
        
        batch_num = np.ceil(len(self.sent) / batch_size)
        batch_X = []
        batch_Y = []
        
        for b in range(batch_num):
            batch_X.append(self.sent[b*batch_size:(b+1)*batch_size])
            batch_Y.append(self.label[b*batch_size:(b+1)*batch_size])
        
        return batch_X, batch_Y

In [ ]:
test_data = Data(idxs=[i for i in range(40000)], is_test=True)
test_data.readData("testset.csv")

In [ ]:
test_data.title[0], test_data.sent[0], len(test_data.sent)

('Efficient Visual Coding: From Retina To V2',
 'The human visual system has a hierarchical structure consisting of layers ofprocessing, such as the retina, V1, V2, etc. Understanding the functional rolesof these visual processing layers would help to integrate thepsychophysiological and neurophysiological models into a consistent theory ofhuman vision, and would also provide insights to computer vision research. Oneclassical theory of the early visual pathway hypothesizes that it serves tocapture the statistical structure of the visual inputs by efficiently codingthe visual information in its outputs. Until recently, most computationalmodels following this theory have focused upon explaining the receptive fieldproperties of one or two visual layers. Recent work in deep networks haseliminated this concern, however, there is till the retinal layer to consider.Here we improve on a previously-described hierarchical model Recursive ICA(RICA) [1] which starts with PCA, followed by a layer o

## Fine-Tune Bert

In [ ]:
import math

# class gelu(nn.Module):
    
#     def __init__(self):
#         super(gelu, self).__init__()

#     def forward(self, x):
#         cdf = 0.5 * (1.0 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))
#         return x * cdf

    
class MultiLabelModel(nn.Module):
    
    def __init__(self, encoder, emb_size=1024, out_size=4, ce_size=23, hidden=256): # hidden=256
        super(MultiLabelModel, self).__init__()
        
        self.encoder = encoder
        self.fn_size = emb_size
        
#         self.fc1 = nn.Linear(emb_size, hidden)
#         self.rnn1 = nn.LSTM(emb_size, hidden, 1, batch_first=True, bidirectional=True) # False
#         self.rnn2 = nn.LSTM(hidden, hidden, 1, batch_first=True, bidirectional=True)
#         self.softmax = nn.Softmax(dim=1)
        self.out_fn = nn.Sequential(
            nn.Dropout(0.35),
            nn.Linear(self.fn_size, self.fn_size//4),
            nn.GELU(),
            nn.Dropout(0.35),
            # nn.Linear(self.fn_size//2, self.fn_size//8),
            nn.Linear(self.fn_size//4, out_size),
        )
        
    def forward(self, inp, seg_inp, cat_emb=None, cls_loc=0): # , inp_title, seg_inp_title, cls_loc=0):

        # batch = 1
        embs = self.encoder(inp, token_type_ids=seg_inp)[0] # [batch, seq, hidden]
#         embs_title = self.encoder(inp_title, token_type_ids=seg_inp_title)[0] # [batch, seq, hidden]
        outputs = embs[:, cls_loc, :]
#         outputs = torch.cat((cat_emb, outputs), 1)
#         outputs_title = embs_title[:, cls_loc, :]
#         outputs = torch.cat((outputs, outputs_title), 1)
#         embs_title = self.encoder(inp_title, token_type_ids=seg_inp_title)[0]
#         outputs, _ = self.rnn1(embs)
#         outputs_title, _ = self.rnn1(embs_title)
#         outputs = outputs.transpose(0, 1)[cls_loc]
#         outputs_title = outputs_title.transpose(0, 1)[cls_loc]
#         outputs = torch.cat((outputs, outputs_title), 1)

#         emb_title = self.encoder(inp_title, token_type_ids=seg_inp_title)[0][:, cls_loc, :] # [batch, emb]
#         emb_title = self.fc1(emb_title) # [batch, hidden]
#         outputs, _ = self.rnn1(embs) # [batch, seq, hidden]
#         attention = torch.mm(emb_title, outputs.squeeze(0).transpose(0, 1)) # [batch, seq]
#         attention = self.softmax(attention).transpose(0, 1) # [seq, batch]
#         outputs = outputs.squeeze(0) * attention # [seq, hidden]
#         outputs, _ = self.rnn2(outputs.unsqueeze(0)) # [batch, seq, hidden]
#         outputs = outputs.transpose(0, 1)[cls_loc] # [batch, hidden*2]
        outputs = self.out_fn(outputs)
#         outputs = torch.cat((cat_emb, outputs), 1)
#         outputs = self.out_fn2(outputs)
        return outputs

In [ ]:
def micro_f1_score(pred, label):
    
    TP = torch.mul(pred, label)[0]
    FP = (torch.mul(pred, (label-1)) != 0)[0]
    FN = (torch.mul(pred-1, label) != 0)[0]
    
    precision = TP.sum() / (TP.sum() + FP.sum())
    recall = TP.sum() / (TP.sum() + FN.sum())
    
    f1 = 2 * precision * recall / (precision + recall)
    
    return f1, TP, FP, FN

In [ ]:
# %%time
good_seed = [5422, 6550, 6545, 9436, 5487, 1070, 7429]
take = 4
Target = -5
train = []
test = []
seed = int(1282) # tune this only
torch.manual_seed(seed)
best_f1 = -1
MAX_LEN = 512
pos_weight = torch.FloatTensor([1., 1., 1.75, 7.5]).to(device)

batch_size = 32
epochs = 5
lr = 1e-5

# for sss
x = [row[:Target] for row in raw_data]
y = [row[Target] for row in raw_data]

# sss
splits = StratifiedShuffleSplit(n_splits=1, test_size=0.07, random_state=seed)
for tr, te in splits.split(x, y):
    train, test = tr, te
# folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
# for i, (tr, te) in enumerate(folds.split(x, y)):
#     if i == 7:
#         train, test = tr, te
#         break

In [ ]:
# read train and dev data
train_data = Data(idxs=train, is_test=False)
train_data.readData("trainset.csv")

dev_data = Data(idxs=test, is_test=False)
dev_data.readData("trainset.csv")

print("train len:{}\ttest len:{}".format(len(train_data.sent), len(dev_data.sent)))
print("train ratios:", end=" ")
for i in range(4):
    print(sum([x[i] for x in train_data.label])/len(train_data.label), end="\t")
print("")
print("dev ratios:", end="   ")
for i in range(4):
    print(sum([x[i] for x in dev_data.label])/len(dev_data.label), end="\t")


train len:6509	test len:491
train ratios: 0.45951759102780765	0.4847134736518666	0.30557689353203255	0.037025656782915965	
dev ratios:   0.46028513238289204	0.4847250509164969	0.3054989816700611	0.03665987780040733	

In [ ]:
# load pre-trained bert
print("\nloading bert...")
tokenizer = BertTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
encoder = BertModel.from_pretrained('allenai/scibert_scivocab_uncased')
# encoder.load_state_dict(torch.load("./model/task1/encoder_{}_state.ckpt".format(9487), map_location=device))


loading bert...


In [ ]:
# init model
print("initialize model...")
model = MultiLabelModel(encoder, emb_size=768, out_size=take)
# model.load_state_dict(torch.load("./model/task2/model_{}_state".format(seed), map_location=device))
# print(1/0)

num_total_steps = np.ceil(len(train_data.sent) / batch_size)*epochs
num_warmup_steps = int(num_total_steps * 0.5)

optim = AdamW(model.parameters(), lr=lr, correct_bias=False)  # To reproduce BertAdam specific behavior set correct_bias=False
scheduler = get_linear_schedule_with_warmup(optim, num_warmup_steps=num_warmup_steps, num_training_steps=num_total_steps)

if use_cuda:
    model = model.to(device)

initialize model...


In [ ]:
# train and validation
print("start training!")
for ep in range(epochs):
    model = model.train()
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    thrld = np.ones((1,take))*0.5
    thrld[0][0] = 0.35
    thrld[0][1] = 0.3
    thrld[0][2] = 0.25
    thrld[0][3] = 0.35
    total_loss = 0.0

    train_metric = { "TP":np.zeros(take), "FP":np.zeros(take), "FN":np.zeros(take), "F1":[] }
    if use_cuda:
        thrld_ten = torch.from_numpy(thrld).float().to(device)
        train_metric["TP"] = torch.from_numpy(train_metric["TP"]).float().to(device)
        train_metric["FP"] = torch.from_numpy(train_metric["FP"]).float().to(device)
        train_metric["FN"] = torch.from_numpy(train_metric["FN"]).float().to(device)
    else:
        thrld_ten = torch.from_numpy(thrld).float()

    optim.zero_grad()
    pbar = enumerate(tqdm(train_data.sent))
    
    for sidx, s in pbar:

        if (sidx+1) % batch_size == 0:
            optim.step()
            scheduler.step()
            optim.zero_grad()
        # if sidx >= 500: break
        
        # abstract
        # indexed_tokens = tokenizer.add_special_tokens_single_sequence(tokenizer.encode(s))[:MAX_LEN]
        indexed_tokens = tokenizer.encode(s)[:MAX_LEN]
        input_ids = torch.tensor([indexed_tokens]).long()
        if use_cuda:
            input_ids = input_ids.to(device)

        segments_ids = [ 0 for i in range(len(indexed_tokens)) ]
        segments_ids = torch.tensor([segments_ids]).long()
        segments_ids = segments_ids.to(device)
        target = torch.FloatTensor(train_data.label[sidx][:take]).to(device).view(1,-1)

        out = model(input_ids, segments_ids) # , category_emb)

        l = criterion(out, target)
        total_loss += l.item()

        l.backward()

        out = torch.sigmoid(out)
        pred = (out > thrld_ten.expand(target.size())).float()
        f1, tp, fp, fn = micro_f1_score(pred, target)

        train_metric["F1"].append(f1)
        train_metric["TP"] += tp.float()
        train_metric["FP"] += fp.float()
        train_metric["FN"] += fn.float()

    optim.step()
    scheduler.step()

    train_precision_all = train_metric["TP"].sum().item() / (train_metric["TP"].sum().item() + train_metric["FP"].sum().item())
    train_recall_all = train_metric["TP"].sum().item() / (train_metric["TP"].sum().item() + train_metric["FN"].sum().item())
    train_micro_f1 = (2 * train_precision_all * train_recall_all) / (train_precision_all + train_recall_all)

    avg_loss = total_loss / len(train_data.sent)

    print("Train Loss:{}\tmicro_f1:{}".format(avg_loss, train_micro_f1))
    print("micro_f1s:", end="")
    for i in range(take):
        precision = train_metric["TP"][i].item() / (train_metric["TP"][i].item()+train_metric["FP"][i].item()+1e-10)
        recall = train_metric["TP"][i].item() / (train_metric["TP"][i].item()+train_metric["FN"][i].item()+1e-10)
        print("{}".format(2*precision*recall / (precision+recall+1e-10)), end="\t")
    print("")

    # Evaluation
    model = model.eval()
    criterion = nn.BCEWithLogitsLoss()

    dev_loss = 0.0
    dev_metric = { "TP":np.zeros(take), "FP":np.zeros(take), "FN":np.zeros(take), "F1":[] }
    if use_cuda:
        dev_metric["TP"] = torch.from_numpy(dev_metric["TP"]).float().to(device)
        dev_metric["FP"] = torch.from_numpy(dev_metric["FP"]).float().to(device)
        dev_metric["FN"] = torch.from_numpy(dev_metric["FN"]).float().to(device)

    with torch.no_grad():

        for sidx, s in enumerate(tqdm(dev_data.sent)):

            # abstract
            indexed_tokens = tokenizer.encode(s)[:MAX_LEN]
            input_ids = torch.tensor([indexed_tokens]).long()
            if use_cuda:
                input_ids = input_ids.to(device)

            segments_ids = [ 0 for i in range(len(indexed_tokens)) ]
            segments_ids = torch.tensor([segments_ids]).long()
            if use_cuda:
                segments_ids = segments_ids.to(device)
                target = torch.FloatTensor(dev_data.label[sidx][:take]).to(device).view(1,-1)
            else:
                target = torch.FloatTensor(dev_data.label[sidx][:take]).view(1,-1)

            out = model(input_ids, segments_ids) # , category_emb)

            l = criterion(out, target)
            dev_loss += l

            out = torch.sigmoid(out)
            pred = (out > thrld_ten.expand(target.size())).float()
            f1, tp, fp, fn = micro_f1_score(pred, target)

            dev_metric["F1"].append(f1)
            dev_metric["TP"] += tp.float()
            dev_metric["FP"] += fp.float()
            dev_metric["FN"] += fn.float()

            if use_cuda:
                input_ids = input_ids.cpu()
                target = target.cpu()

        dev_precision_all = dev_metric["TP"].sum().item() / (dev_metric["TP"].sum().item() + dev_metric["FP"].sum().item())
        dev_recall_all = dev_metric["TP"].sum().item() / (dev_metric["TP"].sum().item() + dev_metric["FN"].sum().item())
        dev_micro_f1 = (2 * dev_precision_all * dev_recall_all) / (dev_precision_all + dev_recall_all)

        avg_loss = dev_loss / len(dev_data.sent)

        print("Dev Loss:{}\tmicro_f1:{}".format(avg_loss.item(), dev_micro_f1))
        print("micro_f1s:", end="")
        for i in range(take):
            precision = dev_metric["TP"][i].item() / (dev_metric["TP"][i].item()+dev_metric["FP"][i].item()+1e-10)
            recall = dev_metric["TP"][i].item() / (dev_metric["TP"][i].item()+dev_metric["FN"][i].item()+1e-10)
            print("{}".format(2*precision*recall / (precision+recall+1e-10)), end="\t")
        print("\n=====================================")

        if dev_micro_f1 > best_f1:
            best_f1 = dev_micro_f1

torch.save(model.state_dict(), "/content/gdrive/MyDrive/aicup_2020/model/model_{}_state.ckpt".format(seed))

start training!



Train Loss:0.6726680499083583	micro_f1:0.5767128741539687
micro_f1s:0.6523675573852996	0.6581133662402174	0.4858742999005071	0.13264649063699505	



Dev Loss:0.43670207262039185	micro_f1:0.665886483323581
micro_f1s:0.7787234042050611	0.7048192770622297	0.5363984673917882	0.45283018863268065	



Train Loss:0.5437868849624319	micro_f1:0.6625984789835603
micro_f1s:0.7367116069547376	0.7217132953225183	0.5661730319135505	0.32971372158262696	



Dev Loss:0.4081539213657379	micro_f1:0.6977635782747604
micro_f1s:0.7844827585703851	0.7661870503104717	0.5637860081875442	0.47457627114243034	


...


Dev Loss:0.4147368371486664	micro_f1:0.6945716154349248
micro_f1s:0.8027842226875652	0.7655677655183071	0.5528455284126743	0.4333333332898889	



Train Loss:0.43260259945120133	micro_f1:0.7306962341424433
micro_f1s:0.7903691813304712	0.7760665121494048	0.6319845856963857	0.5415549597416395	



Dev Loss:0.42884746193885803	micro_f1:0.6936026936026937
micro_f1s:0.7865707433552508	0.7654784239652939	0.5636743214598786	0.42857142852627556	



Train Loss:0.38550385843174334	micro_f1:0.755815729193179
micro_f1s:0.8100775193298763	0.7950842308262549	0.6544999104926886	0.6688102893413855	



Dev Loss:0.3962878882884979	micro_f1:0.7106875425459496
micro_f1s:0.7560975609256329	0.7643784786145924	0.6098191213992402	0.47058823524659743	


## Test Data Prediction

In [ ]:
test_data = Data(idxs=[i for i in range(40000)], is_test=True)
test_data.readData("testset.csv")
    
# load pre-trained bert
print("loading bert...")
tokenizer = BertTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
    
# MAX_LEN = 512
# seed = int(sys.argv[1])
# take = 4
print("seed:", seed)
# device = torch.device("cuda:"+sys.argv[2])
print("device: {}".format(device))

encoder = BertModel.from_pretrained('allenai/scibert_scivocab_uncased')
model = MultiLabelModel(encoder, 768, take)
model.load_state_dict(torch.load("/content/gdrive/MyDrive/aicup_2020/model/model_{}_state.ckpt".format(seed), map_location=device))
model = model.to(device)
model = model.eval()

test_loss = 0.0
thrld = np.ones((1,4))*0.35
# thrld = np.ones((1,4))*0.5
# thrld[0][0] = 0.35
# thrld[0][1] = 0.3
# thrld[0][2] = 0.25
# thrld[0][3] = 0.35
thrld_ten = torch.from_numpy(thrld).float().to(device)

preds = []
ids = []

with torch.no_grad():

    for sidx, s in tqdm(enumerate(test_data.sent), total=40000):

        # abstract
        indexed_tokens = tokenizer.encode(s)[:MAX_LEN]
        input_ids = torch.tensor([indexed_tokens]).long()
        input_ids = input_ids.to(device)

        segments_ids = [ 0 for i in range(len(indexed_tokens)) ]
        segments_ids = torch.tensor([segments_ids]).long()
        segments_ids = segments_ids.to(device)
        
        out = model(input_ids, segments_ids) # , category_emb)

        out = torch.sigmoid(out)
        pred1 = (out > thrld_ten.expand(torch.Size([1, 4]))).float()
        max_idx = torch.argmax(out, 1, keepdim=True)
        one_hot = torch.FloatTensor(out.shape).to(device)
        one_hot.zero_()
        pred2=one_hot.scatter_(1, max_idx, 1)
        pred=(pred1+pred2>=1).float()
        
        preds.append(pred.cpu().tolist())
        ids.append("T"+str(sidx+1))

preds = [pred[0] for pred in preds]

for i in range(len(preds)):
    for j in range(len(preds[i])):
        preds[i][j] = int(preds[i][j])
        
with open("/content/gdrive/MyDrive/aicup_2020/result_{}.csv".format(seed), "w", newline="") as f:
    w = csv.writer(f)
    w.writerow(["Id", "THEORETICAL", "ENGINEERING", "EMPIRICAL", "OTHERS"])
    for i in range(1, 40001):
        w.writerow([str(i)]+preds[i-1])

loading bert...
seed: 1282
device: cuda


In [ ]:
with open("result_{}.csv".format(seed), "w", newline="") as f:
    w = csv.writer(f)
    w.writerow(["Id", "THEORETICAL", "ENGINEERING", "EMPIRICAL", "OTHERS"])
    for i in range(1, 40001):
        w.writerow([str(i)]+preds[i-1])